In [1]:
5+5

10

In [2]:
import sentencepiece as spm
from pathlib import Path


data_dir = Path("../Data/cleaned_data")

In [ ]:



# تدريب tokenizer للعربي
spm.SentencePieceTrainer.Train(
    input=str(data_dir/"train.norm.ar"),
    model_prefix=str(data_dir/"spm_ar_unigram"),
    vocab_size=32000,
    model_type="unigram",         # ممكن تغيّر لـ bpe لو عاوز
    character_coverage=0.9995,    # يغطي كل الحروف العربية
    bos_id=1, eos_id=2, pad_id=0, unk_id=3
)

# تدريب tokenizer للإنجليزي
spm.SentencePieceTrainer.Train(
    input=str(data_dir/"train.norm.en"),
    model_prefix=str(data_dir/"spm_en_unigram"),
    vocab_size=16000,             # أو 32000
    model_type="unigram",         # ينفع bpe برضه
    character_coverage=1.0,       # الإنجليزية ASCII كلها
    bos_id=1, eos_id=2, pad_id=0, unk_id=3
)

print("✅ تم تدريب التوكنيزر للعربي والإنجليزي")


# try them model

In [14]:
tokenizer_dir = Path("../tokenizer_models")

sp_ar = spm.SentencePieceProcessor(model_file=str(tokenizer_dir/"spm_ar_unigram.model"))
sp_en = spm.SentencePieceProcessor(model_file=str(tokenizer_dir/"spm_en_unigram.model"))


In [15]:
x = "الزمالك احسن نادي في مصر"
print(sp_ar.encode(x, out_type=int))

[18024, 5558, 7280, 3558, 6, 2518]


In [16]:
sp_ar.encode_as_pieces(x)

['▁الزم', 'الك', '▁احسن', '▁نادي', '▁في', '▁مصر']

In [17]:
print(sp_en.encode("The book is nice", out_type=int))

[31, 1264, 23, 627]


# encode all data 

In [19]:
import sentencepiece as spm
from pathlib import Path
import os

# Paths
data_dir = Path("../Data/cleaned_data")
tokenizer_dir = Path("../tokenizer_models")
output_dir = Path("../Data/encoded_data")
os.makedirs(output_dir, exist_ok=True)

# Load trained tokenizers
sp_ar = spm.SentencePieceProcessor(model_file=str(tokenizer_dir/"spm_ar_unigram.model"))
sp_en = spm.SentencePieceProcessor(model_file=str(tokenizer_dir/"spm_en_unigram.model"))

def encode_file(in_file, out_file, tokenizer):
    with open(in_file, "r", encoding="utf-8") as fin, open(out_file, "w", encoding="utf-8") as fout:
        for line in fin:
            ids = tokenizer.encode(line.strip(), out_type=int)
            fout.write(" ".join(map(str, ids)) + "\n")

# Apply encoding for train/valid/test
splits = ["train", "validation", "test"]
for split in splits:
    # Arabic
    encode_file(data_dir/f"{split}.norm.ar", output_dir/f"{split}.ids.ar", sp_ar)
    # English
    encode_file(data_dir/f"{split}.norm.en", output_dir/f"{split}.ids.en", sp_en)
    print(f"✅ {split} encoded and saved as IDs")

✅ train encoded and saved as IDs
✅ validation encoded and saved as IDs
✅ test encoded and saved as IDs
